# Прогнозирование продаж

Компания хочет понимать, какой клиент из базы данных с большой долей вероятности согласится купить предложенное оборудование. 

В нашем доступе датасет, собранный для случайного множества клиентов ('id' – идентификатор клиента), с которыми была попытка коммуникации в одном из каналов ('channel_name').

Качество сделанных предсказаний оценивается по метрике ROC AUC между истинными значениями и значениями, полученными в ходе исследования.

# Цели исследования

Разработка модели склонности (бинарной классификации) к покупке клиентом оборудования после коммуникации с ним в одном из каналов.

# Описание данных

Датасет собран для случайного множества клиентов (`id` – идентификатор клиента), с которыми была попытка коммуникации в одном из каналов (`channel_name`).<br>
Целевая переменная (`target`) равна единице, если после коммуникации с клиентом была продажа
оборудования и нулю если нет.<br>
Поле `period` соответствует месяцу сбора признаков на клиента. Лаг между датой коммуникации и сборкой признаков на клиента.

Файлы: 
- dataset_train.parquet - тренировочный датасет;
- features_oot.parquet - тестовый датасет;
- features_types.json - описание типов признаков;
- *sample_submission.csv *- пример файла с загружаемыми результатами;

Для каждой пары 'id' + 'period' собрано более 2500 признаков.

Названия признаков интерпретируются следующим образом:

```
    <модуль><номер признака><глубина агрегации>_<тип>

```
Если признак построен как агрегат (например сумма за период), то указывается `<глубина агрегации>` в
месяцах, в противном случае ставится 0. Также, в качестве `<глубина агрегации>` может быть запись вида '3d6', что указывает на отношение агрегата за 3 месяца к агрегату за 6 месяцев.

Различные типы признаков (<тип>) описаны ниже:

- flg - флаг (значение 1 или 0)
- ctg - категориальный признак
- num - числовой признак
- dt - дата
- cnt -количество
- sum -сумма
- avg - среднее
- sumpct -персентиль по сумме
- part - доля

В файле `features_types.json` дополнительно записан словарь, где для каждой фичи в соответствие ставится тип из списка (numeric, categorical_int, categorical_string)

# План работы

1. Изучить информацию о данных и подготовить данные;
2. Провести исследовательский анализ данных;
3. Построить и обучить модель. 
4. Проверить качество лучшей модели на тестовой сборке.

## Обзор и предобработка данных

**Импортируем необходимые библиотеки вначале документа (pandas, numpy, matplotlib.pyplot и другие).**

In [1]:
# задаем константы
RANDOM_STATE = 50623

In [2]:
# импорт библиотеки
import warnings
import time
import random

In [3]:
# настраиваем фильтр предупреждений
warnings.filterwarnings("ignore")

In [4]:
# импорт библиотек для анализа
import pandas as pd
import numpy as np

Установка библиотек

In [5]:
# импорт библиотек машинного обучения
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    GridSearchCV)

### Обзор файла JSON

Откроем файл `json` с названиями колонок и типом переменной.

In [6]:
# сохраняем путь к файлу json
path_json = '/kaggle/input/yapr1-hackaton/features_types.json'

In [7]:
# читаем файл json
df_json = pd.read_json(path_json, orient='index')

In [8]:
# переименовываем столбец
df_json.columns = ['d_type']

In [9]:
# сохраняем названия столбцов в список
all_columns = list(df_json.index)

Количество колоннок разного типа.

In [10]:
# распечатаем распределение колонок по типу данных.
print(df_json['d_type'].value_counts())

d_type
numeric            2607
categorical_int     138
categorical_str      31
Name: count, dtype: int64


In [11]:
# удаляем файл для экономии памяти
del df_json

### Обзор основного датасета

Создадим функцию `data_review` для автоматизации предобработки данных. В качестве параметров она получает список колонок(`columns`) и путь к файлу(`file_path`),а также флаг `is_train`. При значении `True` из датасета отбирается 1/7 строк для ускорения подбора параметров.<br>

In [12]:
# Изменение ограничения на количество выводимых рядов
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 170)

In [13]:
# сохраняем путь к файлу
data_path = '/kaggle/input/yapr1-hackaton/dataset_train.parquet'

In [14]:
# 
def data_review(columns, file_path, is_train=False):
    data_frame = pd.read_parquet(file_path, 
                       columns=columns)
    if is_train == True:
        data_frame = data_frame.sample(int(data_frame.shape[0]/7), random_state=RANDOM_STATE)

    # получаем общую информацию о датасете
    data_frame['channel_name'] = data_frame['channel_name'].astype('int')

    print('Review completed.', f'Shape: {data_frame.shape}', '-'*115, sep='\n')
    # возвращаем датасет
    return data_frame

**Вывод:**
1. Импортировали необходимые библиотеки.
2. Считали данные из parquet-файла.
3. Вывели общую информацию по датафрейму: 
    
    - в датафрейме 2776 столбцов и 702086 строки;
    - в датасете присутствуют пропуски.
    
    
4. Подготовились к этапу 'Предобработка данных'.
 - изменили тип данных некоторых признаков ('channel_name' и др.),

 На этапе предобработки данных предполагается:
 - оценить и принять решение, как обработать пропуски,
 - оценить мультиколлинеарность в признаках,
 - оценить и принять решение об устранении выбросов,
 - объединить таблицы в одну для этапа "Исследовательский анализ данных".


5. Из предварительного обзора видно, что почти все данные были приведены к типу float, в том числе и строковые, а затем Нормализованы или закодированы. Это усложнило возможность вникнуть в суть признаков и логически обработать признаки. Также были признаки, где количество пропусков достигало 90%.

Поэтому весь обзор данных скрыт и предобработка проводится автоматически.

---
---

## Предобработка данных

### Пропуски

В первую очередь избавимся от пропусков, где это целесообразно. Везде, где доля пропусков меньше 75%, они заменялись медианной, вычисленной по `channel_name` и `period`. Остальные пропуски оставим без изменений. Для автоматизации написана функция `fillna_median`, которая получает датасет(`dataset`), список колонок, которые не обрабатываются(`col_ignor`). После функция возвращает обработанный датасет.

In [15]:
# функция для обработки пропусков
def fillna_median(dataset, col_ignor):
    # в цикле перебираем каналы
    for channel in dataset['channel_name'].unique():
        # в цикле перебираем периоды
        for period in dataset['period'].unique():
            # в цикле перебираем столбцы, где менее 75% пропусков
            for column in (dataset.drop(col_ignor, axis=1)
                                 .drop(columns=dataset.columns[round(dataset.isna().mean()*100,2) > 75])
                                 .columns):
                # отбираем строки с пропусками и заменяем медианой
                dataset.loc[(dataset[column].isna()) &
                            (dataset['channel_name']==channel) &
                            (dataset['period']==period), column] = (
                            dataset.loc[(dataset['channel_name']==channel) &
                            (dataset['period']==period), column].median())
    
    print('Filling completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    # возвращаем датасет
    return dataset

---
---

### Мультиколлинеарность

Далее избавимся от мультиколлинеарности признаков. Для этого напишем функцию `drop_corr`. На вход функция получает датасет(`dataset`), список столбцов, которые не обрабатываются(`col_ignor`) и флаг(`debug_info`), который позволяет получать информацию о ходе обработки и вывести матрицу корреляций. Функция удаляет признаки, у которых стандартное отклонение меньше 0.02, т.е. столбцы в которых большинство значений повторяются.<br>
Далее получена матрица корреляций и на её основе удалены столбцы с корреляцией более 0.85. В конце функция возвращает обработанный датасет.

In [16]:
# функция для устранения мульти коллинеарности
def drop_corr(dataset, col_ignor, debug_info=False):
    # вывод отладочной информации
    if debug_info == True:
        # размер датасета
        print(dataset.shape)
        # вывод описательной статистики
        dataset.describe()
    # получение описательной статистики
    stat_1 = dataset.drop(col_ignor, axis=1).describe().T
    # удаление стобцов с низкой дисперсией
    dataset = dataset[list(stat_1[stat_1['std']>0.02].index)+col_ignor]
    # вывод отладочной информации
    if debug_info == True:
        # размер датасета
        print(dataset.shape)
    # получение матрицы корреляций
    corr_1 = dataset.corr(method='spearman')
    # создаем список удаляемых столбцов
    del_col = set()
    # в цикле пербирем столбцы
    for col in corr_1.columns:
        # чтобы не удалить оба столбца, проверяем что столбца нет в списке
        if col not in del_col:
            # удаляем обрабатываемый столбец, чтобы не удалить,
            # берем модуль от корриляции
            temp = corr_1[col].drop(col, axis=0).abs()
            # удаляем столбцы с большой корреляцией
            del_col.update(set((temp[(temp > 0.85)].index)))
    # вывод отладочной информации
    if debug_info == True:
        # список удаляемых столбцов
        print(del_col)
    # удаляем столбцы
    dataset = dataset.drop(del_col.difference(set(col_ignor)), axis=1)
    # вывод отладочной информации
    if debug_info == True:
        # печатаем матрицу корреляций
        display(dataset.corr(method='spearman')
                .style.background_gradient(cmap='Reds'))
        # размер датасета
        print(dataset.shape)
    print('drop_corr completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    return dataset

---
---

### Устранение выбросов

Для устранения выбросов напишем функцию `fill_outliers`, которая получает датасет(`dataset`) и список столбцов, которые не обрабатываются(`col_ignor`). Для каждого столбца вычисляется верхняя(`upper_whiskers`) и нижняя(`lower_whiskers`) границы нормального расрпеделения. Далее все значения, которые находятся за пределами нормального распределения, заполняются ими. В конце функция возвращает обработанный датасет.

In [17]:
# функция для обработки выбросов
def fill_outliers(dataset, col_ignor):
    # в цикле переберем все столбцы, кроме date
    for column in dataset.drop(col_ignor, axis=1).columns:
        Q1 = dataset[column].quantile(0.25) # 1-й квартиль
        Q3 = dataset[column].quantile(0.75) # 3-й квартиль
        IQR = Q3 - Q1 # межквартильный размах
        upper_whiskers = Q3 + 1.5*IQR # верхняя граница
        lower_whiskers = Q1 - 1.5*IQR # нижняя граница
        # избавимся от выбросов
        dataset.loc[dataset[column] < lower_whiskers, column] = \
        lower_whiskers
        dataset.loc[dataset[column] > upper_whiskers, column] = \
        upper_whiskers
    print('fill_outliers completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    # возвращаем датасет
    return dataset

## Обучение модели

### Разделение на выборки

Напишем функцию `data_split` для разделения данных на выборки. На вход она получает датасет(`dataset`) и возвразает датасет с признаками без `id`,`target` и датасет с целевой переменной `target`.

In [18]:
# функция для разделения на выборки
def data_split(dataset):
    # сохраним в features все столбцы, кроме id, period, target
    # сохраним в target целевой признак
    return (dataset.drop(['id', 'target'], axis=1),
           dataset['target'])

Напишем функцию для автоматизации обучения модели. На вход она получает датасет(`dataset`) и флаг `is_test`, от которого зависит набор перебираемых параметров. Внутри вызывается функция `data_split` для разделения на выборки. Для обучения используется `LGBMClassifier` с параметрами `class_weight` для автобаланса веса классов и `device_type='GPU'` для использования `GPU`, используемый тип бустинга(`boosting_type`) `goss`(обуспечивает более быстрое обучение).<br>
Остальные параметры перебираются с помощью модуля `RandomizedSearchCV` с метрикой(`scoring`) `ROC-AUC` и использованием кросс-валидации(`cv`=10). Перебирались следующие гиперпараметры:
* `n_estimators` - количество деревьев в ансамбле.
* `max_depth` - максимальная глубина деревьев в ансамбле.
* `learning_rate` - шаг обучения.
* `num_leaves` - количество листьев.
* `reg_alpha` - коэффициент регуляризации `l1`.
* `reg_beta` - коэффициент регуляризации `l2`.

In [19]:
# функция для обучения моделей
def lgbm_train(dataset, is_test=False):
    # разделение данных на выборки
    X_train, y_train = data_split(dataset)
    # random_state не перебирается, задаём его прямо в модели
    # создание классификатора LightGBM
    model_lgbm = LGBMClassifier(verbose=-1, random_state=RANDOM_STATE,
                                class_weight = 'balanced',
                                device_type='GPU',
                                num_gpu=512
                               )

    # словарь с гиперпараметрами и значениями при подборе фичей
    if is_test == False:
        param_grid_lgbm = {
            'boosting_type': ['goss'],
            'n_estimators': [20, 35, 60],
            'max_depth': [2, 4, 7],
            'random_state': [RANDOM_STATE],
            'learning_rate': [0.05, 0.2],
            'force_col_wise': [True],
            'num_leaves': [20, 31, 51, 70],
            'reg_alpha': [0, 0.05, 0.4],
            'reg_lambda': [0, 0.05, 0.4]
        }
    # словарь с гиперпараметрами и значениями при обучении финальной модели
    else:
        param_grid_lgbm = {
            'boosting_type': ['goss'],
            'n_estimators': [50, 70, 100, 120],
            'max_depth': [3, 7, 12],
            'random_state': [RANDOM_STATE],
            'learning_rate': [0.1, 0.02, 0.01],
            'force_col_wise': [True],
            'num_leaves': [30, 50, 71],
            'reg_alpha': [0, 0.2, 0.4, 0.8],
            'reg_lambda': [0, 0.2, 0.4, 0.8]
        }
    # создадим объект GridSearchCV
    rs_lgbm = RandomizedSearchCV(
        model_lgbm, 
        param_distributions=param_grid_lgbm, 
        scoring='roc_auc',
        cv = 10,
        n_jobs=-1
    )
    # обучим модель
    rs_lgbm.fit(X_train, y_train, verbose=False)

    # лучшее значение ROC_AUC на кросс-валидации
    print(f'best_score: {rs_lgbm.best_score_}')

    # лучшие гиперпараметры
    print(f'best_params: {rs_lgbm.best_params_}')
    
    # выводим 5 самых важных признаков
    features_imp = pd.DataFrame(data=rs_lgbm.best_estimator_.feature_name_,
                           columns=['name'])
    features_imp['value'] = list(rs_lgbm.best_estimator_.feature_importances_)
    display(features_imp.sort_values(by='value', ascending=False).head())
    
    values = list(rs_lgbm.best_estimator_.feature_importances_)
    # сохраняем перечень наиболе важных признаков, 
    # порог более importance более 15% от максимума
    imp_features = (pd.Series(rs_lgbm.best_estimator_.feature_name_)
                    [values > 0.15*max(values)].to_list())
    print(imp_features)
    print('Trainning completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    
    # возвращаем список важных признаков
    if is_test == False:
        return imp_features
    else:
        return imp_features

---
---

Далее в цикле перебираем по 200 признаков, добавляя их к отобранным ранее. К каждому набору применяются функции описанные выше. Данный цикл повторяется 3 раза, после перемешивания признаков.

In [20]:
t1 = time.perf_counter()
# создаем множество для хранения признаков
imp_features = set()
# количество перебираемых признаков
batch_size = 200

for i in range(3):
    # перебираем по 200 признаков + отобранные ранее
    for i in range(0, (2775 - batch_size + 3), batch_size):
        temp_columns = (list(imp_features.union(['id', 'period', 'channel_name'] +
                                                all_columns[i : i + batch_size] + ['target']))
             )
        
        # печатаем номера обрабатываемых признаков
        print(f'Dataset_{int(1+i/batch_size)}\ncolumns: {i+1}:{i+batch_size+1}')
        # создаем датафрейм
        temp_df = pd.DataFrame()
        # сохраняем данные в датафрейм
        temp_df = data_review(temp_columns, data_path, is_train=True)
        # устраняем мулти коллинеарность
        temp_df = drop_corr(temp_df, ['id', 'period', 'target'], debug_info=False)
        #print('Shape: ', temp_df.shape)
        # заполняем пропуски
        temp_df = fillna_median(temp_df, ['id', 'period', 'target'])
        # устраняем выбросы
        temp_df = fill_outliers(temp_df, ['id', 'period', 'target', 'channel_name'])
        #print('Shape: ', temp_df.shape)
        
        # метка времени перед началом обучения
        t2 = time.perf_counter()
        # обучаем модель и сохраняем отобранные признаки
        imp_features = imp_features.union(set(lgbm_train(temp_df)))
        # метка после обучения
        t3 = time.perf_counter()
        print(f'fit time: {t3-t2:.4f} s')
        # удаляем не используемую переменную
        del temp_df
        print('-'*345)
    # перемешиваем список столбцов
    random.shuffle(all_columns)
    
print(f'Total time: {t3 - t1}')

Dataset_1
columns: 1:201
Review completed.
Shape: (100298, 204)
-------------------------------------------------------------------------------------------------------------------
drop_corr completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------
Filling completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------
fill_outliers completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. P

best_score: 0.6939456648439382
best_params: {'reg_lambda': 0, 'reg_alpha': 0.05, 'random_state': 50623, 'num_leaves': 70, 'n_estimators': 35, 'max_depth': 2, 'learning_rate': 0.2, 'force_col_wise': True, 'boosting_type': 'goss'}


,name,value
26,channel_name,23
135,markers_40_1_cnt,12
109,markers_4_1_cnt,10
74,markers_184_1_cnt,9
84,markers_199_1_cnt,5


['markers_60_1_cnt', 'markers_122_1_cnt', 'channel_name', 'markers_184_1_cnt', 'markers_199_1_cnt', 'markers_4_1_cnt', 'markers_72_1_cnt', 'markers_40_1_cnt', 'markers_104_1_cnt']
drop_corr completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------
fit time: 261.2541 s
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Dataset_2
columns: 201:401
Review completed.
Shape: (100298, 212)
-------------------------------------------------------------------------------------------------------------------
drop_corr completed.
Shape: (100298, 207)
-----------------------------------------------------------------------

...

['traffic_details_65_3_sum', 'channel_name', 'materials_details_4_1_dt', 'charges_details_6_1_sum', 'traffic_details_16_1d3_std', 'campaigns_395_6_part', 'communication_availability_40_1_ctg', 'info_house_5_0_num', 'markers_508_1_cnt', 'traffic_details_61_1_std', 'spas_symptoms_agr_154_12_sum', 'communication_availability_30_1_flg', 'markers_122_1_cnt', 'tariff_plans_4_1_num', 'charges_details_12_1_sum', 'tariff_plans_19_src_id', 'campaigns_400_1d6_part', 'campaigns_394_3d6_part', 'traffic_details_35_1d6_std', 'info_house_6_0_num', 'issues_138_3d6_sum', 'communication_availability_10_1_ctg', 'materials_details_16_1_ctg', 'markers_772_1_cnt']
drop_corr completed.
Shape: (100298, 236)
-------------------------------------------------------------------------------------------------------------------
fit time: 326.7689 s
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. P

best_score: 0.7494705980563919
best_params: {'reg_lambda': 0.4, 'reg_alpha': 0, 'random_state': 50623, 'num_leaves': 51, 'n_estimators': 60, 'max_depth': 2, 'learning_rate': 0.2, 'force_col_wise': True, 'boosting_type': 'goss'}


,name,value
9,channel_name,16
17,charges_details_6_1_sum,7
209,materials_details_16_1_ctg,7
130,tariff_plans_4_1_num,5
92,info_house_5_0_num,5


['materials_details_19_1_dt', 'traffic_details_65_3_sum', 'channel_name', 'materials_details_4_1_dt', 'charges_details_6_1_sum', 'traffic_details_16_1d3_std', 'movix_channels_103_3_sum', 'markers_199_1_cnt', 'campaigns_395_6_part', 'communication_availability_40_1_ctg', 'payments_details_48_3_sum', 'campaigns_354_1_sum', 'info_house_5_0_num', 'traffic_details_61_1_std', 'basic_info_1_0_max', 'spas_symptoms_agr_154_12_sum', 'communication_availability_30_1_flg', 'tariff_plans_4_1_num', 'campaigns_378_6_cnt', 'campaigns_400_1d6_part', 'campaigns_394_3d6_part', 'user_lifetime_3_0_dt', 'info_house_6_0_num', 'communication_availability_10_1_ctg', 'materials_details_16_1_ctg']
drop_corr completed.
Shape: (100298, 230)
-------------------------------------------------------------------------------------------------------------------
fit time: 304.6851 s
---------------------------------------------------------------------------------------------------------------------------------------------

**Вывод:**
1. Подобрали признаки для основного расчета.
2. Лучшая метрика 0,749.
3. Параметры при которых получена наилучшая метрика: 
    
    - 'reg_lambda': 0.4, 
    - 'reg_alpha': 0, 
    - 'random_state': 50623, 
    - 'num_leaves': 51, 
    - 'n_estimators': 60, 
    - 'max_depth': 2, 
    - 'learning_rate': 0.2, 
    - 'force_col_wise': True, 
    - 'boosting_type': 'goss'
    

---
---

### Обучение итоговой модели

Сохраним в `final_columns` список отобранных признаков.

In [21]:
final_columns = list(set(imp_features).union(['id', 'period', 'channel_name', 'target']))

Создаем датафрейм для обучения финального датасета.

In [22]:
final_df = pd.DataFrame()

Сохраняем данные в датафрейм.

In [23]:
final_df = data_review(final_columns, data_path)

Review completed.
Shape: (702086, 95)
-------------------------------------------------------------------------------------------------------------------


Заполняем пропуски.

In [24]:
final_df = fillna_median(final_df, ['id', 'period', 'target'])

Filling completed.
Shape: (702086, 95)
-------------------------------------------------------------------------------------------------------------------


Устраняем выбросы.

In [25]:
final_df = fill_outliers(final_df, ['id', 'period', 'target', 'channel_name'])

fill_outliers completed.
Shape: (702086, 95)
-------------------------------------------------------------------------------------------------------------------


In [26]:
print(final_df.shape)

(702086, 95)


Избавляемся от мультиколлинеарности.

In [27]:
final_df = drop_corr(final_df, ['id', 'period', 'target'], debug_info=True)

(702086, 95)
(702086, 89)
{'spas_symptoms_agr_159_3_std', 'payments_details_45_1_avg', 'user_lifetime_3_0_dt', 'spas_symptoms_int_6_1_cnt', 'traffic_details_12_1_avg', 'spas_symptoms_int_42_1_cnt'}


(702086, 83)
drop_corr completed.
Shape: (702086, 83)
-------------------------------------------------------------------------------------------------------------------


In [28]:
print(final_df.shape)

(702086, 83)


Обучаем финальную модель.

In [29]:
# метка времени перед началом обучения
t9 = time.perf_counter()
best_param = lgbm_train(final_df, is_test=True)
# метка после обучения
t10 = time.perf_counter()
print(f'fit time: {t10-t9:.4f} s')

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGB

best_score: 0.7509748308427132
best_params: {'reg_lambda': 0, 'reg_alpha': 0, 'random_state': 50623, 'num_leaves': 50, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'force_col_wise': True, 'boosting_type': 'goss'}


,name,value
23,channel_name,58
24,materials_details_4_1_dt,26
6,communication_availability_30_1_flg,25
26,arpu_0_1_sum,23
57,basic_info_1_0_max,21


['traffic_details_35_1d6_std', 'user_lifetime_2_1_num', 'payments_details_28_3_sumpct', 'communication_availability_30_1_flg', 'markers_122_1_cnt', 'materials_details_19_1_dt', 'payments_details_50_6_sum', 'markers_904_1_cnt', 'campaigns_378_6_cnt', 'balance_details_0_1_num', 'charges_details_12_1_sum', 'channel_name', 'materials_details_4_1_dt', 'arpu_0_1_sum', 'traffic_details_16_1d3_std', 'tariff_plans_21_1_max', 'info_house_6_0_num', 'markers_40_1_cnt', 'traffic_details_14_1_sum', 'info_house_5_0_num', 'spas_symptoms_agr_162_6_std', 'communication_availability_10_1_ctg', 'markers_324_1_cnt', 'spas_symptoms_agr_106_12_sum', 'markers_508_1_cnt', 'materials_details_16_1_ctg', 'basic_info_1_0_max', 'movix_channels_103_3_sum', 'spas_symptoms_agr_154_12_sum', 'tariff_plans_18_1_ctg', 'user_active_24_0_dt', 'markers_772_1_cnt', 'campaigns_395_6_part', 'spas_symptoms_agr_115_6_sum', 'area_0_0_num']
drop_corr completed.
Shape: (702086, 83)
---------------------------------------------------

## Предсказание на тестовой выборке

Загружаем тестовую выборку.

In [30]:
# Добавляем id и period
df_test_1 = pd.read_parquet('/kaggle/input/yapr1-hackaton/features_oot.parquet', 
                            columns=final_df.drop('target', axis=1).columns)

In [31]:
print(df_test_1.shape)

(60661, 82)


In [32]:
# изменяем тип колонки на int
df_test_1['channel_name'] = df_test_1['channel_name'].astype('int')

Заполняем пропуски.

In [33]:
df_test_1 = fillna_median(df_test_1, ['id', 'period'], is_test=True)

Filling completed.
Shape: (60661, 82)
-------------------------------------------------------------------------------------------------------------------


Устраняем выбросы.

In [34]:
df_test_1 = fill_outliers(df_test_1, ['id', 'period', 'channel_name'])

fill_outliers completed.
Shape: (60661, 82)
-------------------------------------------------------------------------------------------------------------------


In [35]:
print(final_df.shape, df_test_1.shape, sep='\n')

(702086, 83)
(60661, 82)


Список отобранных параметров.

In [36]:
print(final_df.columns.to_list())

['spas_symptoms_int_17_1_cnt', 'charges_details_23_6_avg', 'traffic_details_35_1d6_std', 'user_lifetime_2_1_num', 'payments_details_19_1d6_avg', 'payments_details_28_3_sumpct', 'communication_availability_30_1_flg', 'markers_60_1_cnt', 'markers_122_1_cnt', 'spas_symptoms_int_108_1_cnt', 'materials_details_19_1_dt', 'payments_details_46_1_sum', 'campaigns_396_6_sum', 'payments_details_34_3_sum', 'payments_details_50_6_sum', 'markers_904_1_cnt', 'communication_availability_40_1_ctg', 'tariff_plans_4_1_num', 'campaigns_378_6_cnt', 'markers_4_1_cnt', 'balance_details_0_1_num', 'traffic_details_9_3d6_part', 'charges_details_12_1_sum', 'channel_name', 'materials_details_4_1_dt', 'tariff_plans_19_src_id', 'arpu_0_1_sum', 'traffic_details_20_1d6_sum', 'charges_details_6_1_sum', 'traffic_details_16_1d3_std', 'payments_details_33_1_sum', 'communication_availability_32_1_ctg', 'markers_72_1_cnt', 'payments_details_48_3_sum', 'payments_details_27_1_sumpct', 'tariff_plans_21_1_max', 'campaigns_385_

Функция для разделения на выборки.

In [37]:
def data_split_2(dataset):
    # сохраним в features все столбцы, кроме id, period, target
    # сохраним в target целевой признак
    return (dataset.drop(['id', 'target'], axis=1),
           dataset['target'])

Создаем тестовую выборку.

In [38]:
X_test = df_test_1.drop(['id'], axis=1)

Обучаем финальную модель.

In [39]:
t9 = time.perf_counter()

X_train, y_train = data_split_2(final_df)
# random_state не перебирается, задаём его прямо в модели
model_lgbm = LGBMClassifier(verbose=-1, random_state=RANDOM_STATE,
                            class_weight = 'balanced',
                            #scale_pos_weight=127,
                            device_type='GPU',
                            num_gpu=512
                            )

# словарь с гиперпараметрами и значениями, которые хотим перебрать
param_grid_lgbm = {
    'boosting_type': ['goss'],
    'n_estimators': [30, 50, 70],
    'max_depth': [2, 5, 7, 12],
    'random_state': [RANDOM_STATE],
    'learning_rate': [0.1, 0.05, 0.02, 0.01],
    'force_col_wise': [True],
    'num_leaves': [20, 30, 50, 71],
    'reg_alpha': [0, 0.05, 0.3, 0.8],
    'reg_lambda': [0, 0.05, 0.3, 0.8]
}
# создадим объект GridSearchCV
gs_lgbm = RandomizedSearchCV(
    model_lgbm, 
    param_distributions=param_grid_lgbm, 
    scoring='roc_auc',
    cv = 10,
    n_jobs=-1
)
# обучим модель
gs_lgbm.fit(X_train, y_train, verbose=False)
t10 = time.perf_counter()
# лучшее значение ROC_AUC на кросс-валидации
print(f'best_score: {gs_lgbm.best_score_}')

# лучшие гиперпараметры
print(f'best_params: {gs_lgbm.best_params_}')
print(f'fit time: {t9-t10:.4f} s')

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGB

best_score: 0.749438039299356
best_params: {'reg_lambda': 0.8, 'reg_alpha': 0.05, 'random_state': 50623, 'num_leaves': 20, 'n_estimators': 70, 'max_depth': 7, 'learning_rate': 0.05, 'force_col_wise': True, 'boosting_type': 'goss'}
fit time: -1116.3910 s


In [40]:
# лучшее значение ROC_AUC на кросс-валидации
print(f'best_score: {gs_lgbm.best_score_}')
features_imp = pd.DataFrame(data=gs_lgbm.best_estimator_.feature_name_,
                           columns=['name'])
features_imp['value'] = list(gs_lgbm.best_estimator_.feature_importances_)
display(features_imp.sort_values(by='value', ascending=False))

best_score: 0.749438039299356


,name,value
23,channel_name,100
26,arpu_0_1_sum,60
51,spas_symptoms_agr_106_12_sum,59
6,communication_availability_30_1_flg,57
24,materials_details_4_1_dt,48
22,charges_details_12_1_sum,43
55,materials_details_16_1_ctg,38
57,basic_info_1_0_max,35
45,communication_availability_10_1_ctg,34
38,info_house_6_0_num,32


Получаем предсказание на тестовой выборке.

In [41]:
test_predict =  gs_lgbm.best_estimator_.predict_proba(X_test)

Считываем файл, куда сохраним результаты.

In [42]:
submission = pd.read_csv('/kaggle/input/yapr1-hackaton/sample_submission.csv')
display(submission)

,id,target
0,0,0.343518
1,1,0.591216
2,2,0.913150
3,3,0.560035
4,4,0.352795
...,...,...
60656,60656,0.765319
60657,60657,0.533016
60658,60658,0.784497
60659,60659,0.804431


In [43]:
#Заменяем столбец с данными из примера на предикт
submission['target'] = test_predict[:,1]
display(submission)

,id,target
0,0,0.582337
1,1,0.497435
2,2,0.636259
3,3,0.567971
4,4,0.742927
...,...,...
60656,60656,0.632572
60657,60657,0.743266
60658,60658,0.704438
60659,60659,0.761793


Созраняем файл с результатми.

In [44]:
#Сохраняем данные на гугл диск или локально и потом сабмитим результат
submission.to_csv('/kaggle/working/my_predict.csv', index=False)

## Отчет по работе

При проверке на публичном датасете самая высокая метрика `ROC-AUC`, которую удалось достичь, 0,6473.
Ее удалось добиться достаточно подробным подбором признаков на модели `LightGBM`. 

Также была опробована модель `Catboost`, которая не дала метрику значительно выше, но по времени расчета была гораздо весомее. 

По результатам Хакатона - на скрытой выборке результат получился 0,66. Наша команда поднялась в рейтинге, что говорит о том, что сама модель была устойчивой и не потеряла в качестве. 

Победитель имел метрику 0,675. После разбора решения, пришли к выводу, что очень сильно повлияли на нее значения параметра `'learning_rate'`, который принимался командой победителями 0,01 и ниже, а также размер `'iterations'` равный 1000. 

По принципу подбора признаков каких-то новых идей не было, но все сошлись во мнении, что наиболее удачное количество признаков для расчета это 120-150 признаков. Что также было нами тоже доказано. 

Также хотелось бы отметить, что из-за большого файла данных, из которого брались значения признаков и сами признаки - попробовать посчитать модель `Catboost` на GPU на GoogleColab было невозможно, так как время сессии было гораздо меньше, чем скорость загрузки документа на сайт. В Kaggle данный расчет на GPU тоже подвисал после 40 минут расчетов. 

В качестве рекомендаций для улучшения кода: 

- Провести все этапы предобработки через pipeline;
- Подумать над новыми способами предобработки (подбор признаков и заполнение пропусков);
- Попробовать решить данную задачу математически или через  нейронные сети(хотя применение нейронок в отношении табличных данных сомнительно)